### __LIBRARIES__

In [ ]:
import numpy as np
import os
import pandas as pd
import sys

path = "/Users/migue/Documents/F1 Data Center/F1-Data-Center/Programs/Python/"

sys.path.append(path)
import multidata

# plt.style.use('classic')


__DATA LOADING__

In [ ]:
path = "/Users/migue/Documents/F1 Data Center/Datos/"
os.chdir(path)

df = pd.read_csv('data.csv', sep = ';', encoding = "ISO-8859-1")

display(df)

colNames: list = df.columns.values.tolist() #Column names
print(colNames)

### __YEARS SET__

In [ ]:
year = 2022

#Data of the corresponding day
if year != 'all':
    index_year = df['Year'] == year
    df_Year = df[index_year]
    display(df_Year)
else:
    df_Year = df
    display(df_Year)


### __POINTS BASED ON FREE PRACTICES, QUALIFYING AND RACE (BY YEAR)__ 

In [ ]:
drivers = list(df_Year["Driver"].unique()) #Drivers participating in the corresponding time period
n_gps = list(df_Year["GP"].unique()) #GPs during the corresponding time period


colors = multidata.f1_teams(drivers, year, 'Default', 'Default').color  #Colors of each team deppending 

#Times in each GP and each driver and the corresponding points obtained during that period
fp1_times = np.zeros((len(n_gps),len(drivers)))
fp1_points = np.zeros((len(n_gps),len(drivers)))

fp2_times = np.zeros((len(n_gps),len(drivers)))
fp2_points = np.zeros((len(n_gps),len(drivers)))

fp3_times = np.zeros((len(n_gps),len(drivers)))
fp3_points = np.zeros((len(n_gps),len(drivers)))

quali_times = np.zeros((len(n_gps),len(drivers)))
quali_points = np.zeros((len(n_gps),len(drivers)))

race_pos = np.zeros((len(n_gps),len(drivers)))
race_points = np.zeros((len(n_gps),len(drivers)))

yearData = df_Year.to_numpy()
driver_name = colNames.index("Driver")
gp_index = colNames.index("GP")

fp1 = colNames.index("fp1_fastest_lap")
fp2 = colNames.index("fp2_fastest_lap")
fp3 = colNames.index("fp3_fastest_lap")
quali = colNames.index("quali_fastest_lap")
race = colNames.index("race_Result")


for row in yearData:
    gpName = row[gp_index] #GP on this row
    driv = row[driver_name] #Driver on this row

    i = n_gps.index(gpName)
    j = drivers.index(driv)

    fp1_times[i,j] = row[fp1]
    fp2_times[i,j] = row[fp2]
    fp3_times[i,j] = row[fp3]
    quali_times[i,j] = row[quali]
    if row[race] == 'DNF': race_pos[i,j] = 0
    else:  race_pos[i,j] = row[race]



Compute the order of the drivers in each GP

In [ ]:
points = [25,18,15,12,10,8,6,4,2,1,0,0,0,0,0,0,0,0,0,0]

sesion = [fp1_times.copy(),fp2_times.copy(),fp3_times.copy(),quali_times.copy(), race_pos.copy()] #Include every Sesion time (4, len(gPs), len(drivers))
ses_points = [fp1_points.copy(),fp2_points.copy(),fp3_points.copy(),quali_points.copy(), race_points.copy()] #Include every Sesion points (4, len(gPs), len(drivers))

for ses_index, s in enumerate(sesion):
    for circ_index, circuit in enumerate(s):
        p = -1
        if sum(circuit) > 10:
            while circuit[np.argmin(circuit)] < sys.maxsize:
                minim = np.argmin(circuit)
                if circuit[minim] == 0:
                    circuit[minim] = sys.maxsize
                else:
                    p += 1
                    minim =np.argmin(circuit)
                    circuit[minim] = sys.maxsize
                    ses_points[ses_index][circ_index,minim] = points[p]

# print(ses_points[0][:,drivers.index('Nyck VRIES')])

__Total Points__ 

In [ ]:
titles = ['FP1 TOTAL POINTS', 'FP2 TOTAL POINTS','FP3 TOTAL POINTS', 'QUALIFYING TOTAL POINTS', 'RACE TOTAL POINTS']
totalPoints = np.zeros((len(ses_points),len(drivers))) #Reduce the total to (4, len(driveres)) matrix which incluedes the total amount of points scored by each driver in each Sesion

for ses_index, s in enumerate(ses_points):
    for driv_index, driver in enumerate(drivers):
        totalPoints[ses_index,driv_index] = sum(s[circ_index,driv_index] for circ_index, circuit in enumerate(s))

#print(totalPoints[0])

__Year Progression__

In [ ]:
yearProgessionOK = np.zeros((len(ses_points),len(drivers),len(n_gps))) #Each sesion, and evolution of each driver points per race 

for ses_index, s in enumerate(ses_points):
    for gpInd, gp in enumerate(n_gps):
        for driv_index, driver in enumerate(drivers):
            totalUntilGP = sum(s[circ_index,driv_index] for circ_index in range(gpInd+1))
            yearProgessionOK[ses_index,driv_index, gpInd] = totalUntilGP

#yearProgessionOK[0,0]

__Session Progression__

In [ ]:
sesionProgress = np.zeros((len(drivers),len(totalPoints))) #Each driver contains a list of te points obtained in each season 
for driverInd, d in enumerate(drivers):
    for ses, s in enumerate(totalPoints):
        sesionProgress[driverInd,ses] = totalPoints[ses,driverInd]

#sesionProgress

__Data Organization__

In [ ]:
def orderLists(points: np.array, driverName: list, color: list):
    '''Necessary to plot, orders the list in function on the first introduced list. LISTS MUST HAVE THE SAME SIZE'''
    newPoints, newDriv, newColor = [], [], []
    while points[np.argmax(points)] >= 0:
        maxim =  np.argmax(points)

        newPoints.append(points[maxim])
        newDriv.append(driverName[maxim])
        newColor.append(color[maxim])
            
        points[maxim] = -1

    return newPoints[:22], newDriv[:22], newColor[:22]

__PLOTS__

In [ ]:
#### PLOT SETTINGS BY YEAR ####
import matplotlib.pyplot as plt
# backscreen, legend = multidata.plotColors(year).color
%matplotlib inline 
plt.rcParams["figure.figsize"] = (10.4, 8.8)
plt.rcParams["axes.facecolor"] =  '#DFF0FF'
plt.rcParams["legend.facecolor"] = '#DFF0FF'
plt.rcParams["legend.edgecolor"] = '#000000'
################################

def driv_min(drivers):
    driv_min = []
    for d in drivers:
        surname = d.split()[-1]
        if surname[0:3] not in driv_min:
            driv_min.append(surname[0:3].upper())
        else:
            cond = True
            while cond:
                x = 4
                if surname[0:x] not in driv_min:
                    cond = False
                    driv_min.append(surname[0:3].upper())
                else:
                    x += 1

    return driv_min

def addlabels(y, colors):
    for i, v in enumerate(sorted(y, reverse=True)):
        plt.text(v+0.2, i, str(int(v)), color='#090909', va="center")

drivers_dim = driv_min(drivers.copy())
for s_index,sesion in enumerate(totalPoints):
    ses,drivers_dimPlot,colorTeam = orderLists(sesion.copy(),drivers_dim.copy(),colors.copy())
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1]) 

    ax.barh(drivers_dimPlot,ses,color = colorTeam)
    addlabels(ses, colorTeam)
    
    plt.title(titles[s_index]) 
    plt.xlabel("Points")
    plt.ylabel("Drivers")

    plt.gca().invert_yaxis()
    plt.xticks(range(0, int(ses[0]), 50))
    plt.show()

__During Year Progression Plot__

In [ ]:

def maxValuePlot(sesion):
    sesion = np.array(sesion)
    sesion[sesion == np.inf] = np.nan
    maxim = np.nanmax(sesion)
    return maxim

def minValuePlot(sesion):
    minim = np.nanmin(sesion)
    return minim

def lenTicks(minim:int, maxim:int):
    maxTick = 15
    lenTick = (maxim-minim)/maxTick
    return int(lenTick)

def orderByTeam(yearProgessionOK: np.array, colorsPlot: list, driversPlot: list):
    yearProgession = []
    colors = []
    drivers = []

    yearProgessionTeams = []
    colorsTeams = []
    teams = []
    for s_index,sesion in enumerate(yearProgessionOK):
        ses = []
        col = []
        drivList = []

        sesTeams = []
        colTeams = []
        teamsList = []        


        teamsName = multidata.f1_teams(driversPlot[s_index], year, 'Default', 'Default').equipos
        teamsCol = list(set(colorsPlot[s_index]))
        
        points = [driver[-1] for driv_index, driver in enumerate(sesion)]
        teamIndexes = []
        teamPoints = []

        for t in teamsCol:
            teamIndexes.append([i for i in range(len(colorsPlot[s_index])) if colorsPlot[s_index][i] == t])
            teamPoints.append(sum(points[p] for p in range(len(points)) if colorsPlot[s_index][p] == t))
        
        orderedIndex = []
        originalLen = len(teamPoints)
        while len(orderedIndex) != originalLen:
            index = teamPoints.index(max(teamPoints))
            orderedIndex.append(teamIndexes[index])
            teamPoints[index] = -1

        for t, teamIndexArray in enumerate(orderedIndex):
            total = np.zeros(sesion[0].shape) #MUST BE THE TOTAL NUMBER OF RACES DURING A YEAR
            for d in teamIndexArray:
                #DRIVERS
                ses.append(sesion[d])
                col.append(colorsPlot[s_index][d])
                drivList.append(driversPlot[s_index][d])

                #TEAMS TOTAL
                total += sesion[d]

            #TEAMS
            sesTeams.append(total)
            colTeams.append(colorsPlot[s_index][d])
            teamsList.append(teamsName[d])

        yearProgession.append(ses)
        colors.append(col)
        drivers.append(drivList)

        yearProgessionTeams.append(sesTeams)
        colorsTeams.append(colTeams)
        teams.append(teamsList)

    return yearProgession, colors, drivers, yearProgessionTeams, colorsTeams, teams

#REMOVE 0s drivers for a better visibility
colorsPlot = []
driversPlot = []

yearProgession = []
for s_index,sesion in enumerate(yearProgessionOK):
    ses = []
    col = []
    drivList = []
    for driv_index, driver in enumerate(sesion):
        if driver[-1] != 0: # 0 in the last GP
            ses.append(driver)
            col.append(colors[driv_index])
            drivList.append(drivers[driv_index])

    yearProgession.append(ses)
    colorsPlot.append(col)
    driversPlot.append(drivList)

yearProgession, colorsPlot, driversPlot, yearProgessionTeams, colorsTeams, teams = orderByTeam(yearProgession.copy(), colorsPlot.copy(), driversPlot.copy())

teamsQuali = teams[3] #Used in Qualyfing Section

x_axes = [i for i in range(len(n_gps))]
for s_index,sesion in enumerate(yearProgession):
    teams_av = []
    teams_driv = multidata.f1_teams(driversPlot[s_index], year, 'Default', 'Default').equipos
    drivers_dim = driv_min(driversPlot[s_index].copy())
    for driv_index, driver in enumerate(sesion):
        if teams_driv[driv_index] not in teams_av:
            teams_av.append(teams_driv[driv_index])
            plt.plot(x_axes, driver, linewidth=2 , color = colorsPlot[s_index][driv_index], label = drivers_dim[driv_index]) 
        else:
            plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[s_index][driv_index], label =drivers_dim[driv_index])


    
    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(titles[s_index]) 
    plt.xlabel("GP")
    plt.ylabel("Total Points")
    plt.xticks(range(0, int(len(n_gps))), labels = n_gps, rotation=90)
    plt.yticks(range(0, int(maxValuePlot(sesion) + 25), 25))
    plt.show()

__During Year Teams Progression Plot__

In [ ]:
x_axes = [i for i in range(len(n_gps))]
for s_index,sesion in enumerate(yearProgessionTeams):
    # drivers_dim = driv_min(driversPlot[s_index].copy())
    for team_index, team in enumerate(sesion):
        plt.plot(x_axes, team, linewidth=2 , color = colorsTeams[s_index][team_index], label = teams[s_index][team_index]) 
    
    ax = plt.gca()
    ax.legend(loc=0, bbox_to_anchor=(1, 0.75))
    plt.title(titles[s_index]) 
    plt.xlabel("GP")
    plt.ylabel("Total Points")
    plt.xticks(range(0, int(len(n_gps))), labels = n_gps, rotation=90)
    plt.yticks(range(0, int(maxValuePlot(sesion) + 50), 50))
    plt.show()

__Drivers Progression Plot by Session__

In [ ]:
def orderByTeam(sesionProgessionOK: np.array, colorsPlot: list, driversPlot: list):
    #1. Sum all the results from the drivers of the same team
    for driverInd,driver in enumerate(sesionProgessionOK):   
        allTeams = multidata.f1_teams(driversPlot, year, 'Default', 'Default').equipos

    teams = list(set(allTeams))

    colorsTeams = []
    totalPointsTeams = np.zeros(len(teams))
    sesionProgessionTeam = np.zeros((len(teams), len(sesionProgessionOK[0]))) #Similar format to drivers, n teams and the corresponding m sessions inside
    for t,team in enumerate(teams):
        teamIndex = [t_index for t_index,teamName in enumerate(allTeams) if teamName == team]
        colorsTeams.append(colorsPlot[teamIndex[0]]) #First element that appears in teamIndex
        for index in teamIndex:
            sesionProgessionTeam[t] += sesionProgessionOK[index]
        totalPointsTeams[t] = sesionProgessionTeam[t,4] #4 is corresponding to race
    
    #2. Order teams by points obtained in Race
    sesionProgessionTeamOrdered =  np.zeros_like(sesionProgessionTeam)
    colorsTeamsOrdered = []
    teamsOrdered = []
    orderedTeamIndex =(-totalPointsTeams).argsort() #Sort the points obtained by each team and obtain its index values
    for i,teamIndex in enumerate(orderedTeamIndex):
        sesionProgessionTeamOrdered[i] += sesionProgessionTeam[teamIndex]
        colorsTeamsOrdered.append(colorsTeams[teamIndex])
        teamsOrdered.append(teams[teamIndex])

    #3. Order drivers grouping them by teams
    sesionProgessionOKOrdered = np.zeros_like(sesionProgessionOK)
    colorsPlotOrdered = []
    driversPlotOrdered = []
    defIndex = 0
    for t,team in enumerate(teamsOrdered):
        teamIndex = [t_index for t_index,teamName in enumerate(allTeams) if teamName == team]
        for index in teamIndex:
            sesionProgessionOKOrdered[defIndex] += sesionProgessionOK[index]
            colorsPlotOrdered.append(colorsPlot[index])
            driversPlotOrdered.append(driversPlot[index])
            defIndex += 1
        
    return sesionProgessionOKOrdered, colorsPlotOrdered, driversPlotOrdered, sesionProgessionTeamOrdered, colorsTeamsOrdered, teamsOrdered

#REMOVE 0s drivers for a better visibility
colorsPlot = []
driversPlot = []

sesionProgressNot0 = []
for driv_index,driver in enumerate(sesionProgress):
    if driver[-1] != 0: # 0 points in the Championship
        sesionProgressNot0.append(driver)
        colorsPlot.append(colors[driv_index])
        driversPlot.append(drivers[driv_index])

sesionProgressNot0 = np.array(sesionProgressNot0)

sesionProgressNot0, colorsPlot, driversPlot, sesionProgressTeams, colorsTeams, teams = orderByTeam(sesionProgressNot0, colorsPlot.copy(), driversPlot.copy())


x_axes = [i for i in range(len(sesionProgressNot0[0]))]
teams_av = []
teams_driv = multidata.f1_teams(driversPlot, year, 'Default', 'Default').equipos
drivers_dim = driv_min(driversPlot.copy())
for driv_index, driver in enumerate(sesionProgressNot0):
    if teams_driv[driv_index] not in teams_av:
        teams_av.append(teams_driv[driv_index])
        plt.plot(x_axes, driver, linewidth=2 , color = colorsPlot[driv_index], label = drivers_dim[driv_index]) 
    else:
        plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[driv_index], label =drivers_dim[driv_index])


sessions = ['FP1', 'FP2', 'FP3','Qualifying','Race']
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('DRIVERS SESSION PROGRESS') 
plt.xlabel("Session")
plt.ylabel("Points by Session")
plt.xticks(range(0, int(len(sesionProgressNot0[0]))), labels = sessions, rotation=90)
plt.yticks(range(0, int(maxValuePlot(sesionProgressNot0) + 25), 25))
# plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.show()

__Teams Progression Plot by Session__

In [ ]:
x_axes = [i for i in range(len(sesionProgressNot0[0]))]
for team_index, team in enumerate(sesionProgressTeams):
    plt.plot(x_axes, team, linewidth=2 , color = colorsTeams[team_index], label = teams[team_index]) 

ax = plt.gca()
ax.legend(loc=0, bbox_to_anchor=(1, 0.75))
plt.title('TEAMS SESSION PROGRESS') 
plt.xlabel("Session")
plt.ylabel("Points by Session")
plt.xticks(range(0, int(len(sesionProgressNot0[0]))), labels = sessions, rotation=90)
plt.yticks(range(0, int(maxValuePlot(sesionProgressTeams) + 50), 50))
plt.show()

__Data Preprocessing__

In [ ]:
qualiDiference = np.zeros((len(n_gps),len(drivers)))
teams = list(set(multidata.f1_teams(drivers, year, 'Default', 'Default').equipos))
qualiTeamDiference = np.zeros((len(n_gps),len(teams)))
qualiTeamDiferencePerc = np.zeros((len(n_gps),len(teams)))
            
def my_func(a):
    negArray = np.zeros_like(a)
    for i,elem in enumerate(a):
        if elem == 0:
            negArray[i] = np.inf
        else:
            negArray[i] = elem
    return negArray

def getColorsTeam(drivers: list, teams: list, year: int = year):
    colorsTeams = []
    allTeams = multidata.f1_teams(drivers, year, 'Default', 'Default').equipos
    colors = multidata.f1_teams(drivers, year, 'Default', 'Default').color
    for t in teams:
        colorsTeams.append(colors[allTeams.index(t)])
    return colorsTeams

def qualiTeamsFun(quali_times: np.ndarray, drivers: list, teams: list,year: int = year):
    allTeams = multidata.f1_teams(drivers, year, 'Default', 'Default').equipos
    for t in allTeams:
        if t not in teams: teams.append(t)
    
    qualiTeams = np.zeros(len(teams))
    for tIndex, team in enumerate(teams):
        teamDriversIndeces = [dIndex for dIndex,teamName in enumerate(allTeams) if teamName == team]
        allTimes = 0
        notInf = 0
        for index in teamDriversIndeces:
            if quali_times[index] != np.inf:
                allTimes += quali_times[index]
                notInf += 1
        if allTimes > 0: qualiTeams[tIndex] = allTimes/(notInf)
        else: qualiTeams[tIndex] = np.inf
    return qualiTeams, teams
                            
def removeOutliers(qualiDiference: np.ndarray, drivers: list, limitReturn: tuple):
    limit,ret = limitReturn
    newDrivers = []
    removedIndeces = []
    qualiDiference[(qualiDiference >= limit) & (qualiDiference != np.inf)] = ret #Over 5 Second is understood as an outlier and therefore it does not count.
    for dIndex,driv in enumerate(qualiDiference.T):
        driv = driv.copy()
        driv[driv != np.inf] = False
        driv[driv == np.inf] = True
        if sum(driv) < len(driv):
            newDrivers.append(drivers[dIndex])
        else:
            removedIndeces.append(dIndex)

    qualiDiference = np.delete(qualiDiference, removedIndeces, axis=1)

    return qualiDiference, newDrivers

for gp_index,gp in enumerate(n_gps):
    # Obtain array of square of each element in x
    #Drivers
    negArray = my_func(quali_times[gp_index])
    minimumGP = np.amin(negArray)
    #Teams
    quali_times_teams, teams = qualiTeamsFun(negArray, drivers, teamsQuali)
    minimumGPTeam = np.amin(quali_times_teams)
    for dIndex,driverTime in enumerate(drivers):
        qualiDiference[gp_index,dIndex] = negArray[dIndex] - minimumGP
    for tIndex, team in enumerate(teams):
        qualiTeamDiference[gp_index,tIndex] = quali_times_teams[tIndex] - minimumGPTeam
    for tIndex, team in enumerate(teams):
        qualiTeamDiferencePerc[gp_index,tIndex] = quali_times_teams[tIndex]*100/minimumGPTeam

qualiDiference, driversNotInf = removeOutliers(qualiDiference, drivers, (5000,6000))
qualiTeamDiference, teamsNotInf = removeOutliers(qualiTeamDiference, teams,  (5000,6000))
qualiTeamDiferencePerc, teamsInf = removeOutliers(qualiTeamDiferencePerc, teams, (107,107))

colorsTeams = getColorsTeam(drivers,teams)

__Qualyfing Time Difference per Driver__

In [ ]:
def orderByTeam(qualiDiference: np.ndarray, drivers: list, orderedTeams:list, year = year):
    qualiDiferenceOrdered = np.zeros_like(qualiDiference)
    driversOrdered = []
    driversTeams =  multidata.f1_teams(drivers, year, 'Default', 'Default').equipos
    #Append to ordered teams all teams that does not appear because they obtained 0 points
    for t in driversTeams:
        if t not in orderedTeams: orderedTeams.append(t)

    defIndex = 0
    for tIndex, team in enumerate(orderedTeams):
        teamDriversIndeces = [dIndex for dIndex,teamName in enumerate(driversTeams) if teamName == team]
        for index in teamDriversIndeces:
            qualiDiferenceOrdered[:,defIndex] = qualiDiference[:,index]
            driversOrdered.append(drivers[index])
            
            defIndex += 1
    return qualiDiferenceOrdered, driversOrdered, orderedTeams

qualiDiferenceOrdered, driversPlot, orderedTeams = orderByTeam(qualiDiference, driversNotInf, teamsQuali) 
colorsPlot =  multidata.f1_teams(driversPlot, year, 'Default', 'Default').color

x_axes = [i for i in range(len(n_gps))]
teams_av = []
teams_driv = multidata.f1_teams(driversPlot, year, 'Default', 'Default').equipos
drivers_dim = driv_min(driversPlot.copy())
for driv_index, driver in enumerate(qualiDiferenceOrdered.T):
    if teams_driv[driv_index] not in teams_av:
        teams_av.append(teams_driv[driv_index])
        plt.plot(x_axes, driver, linewidth=2 , color = colorsPlot[driv_index], label = drivers_dim[driv_index]) 
    else:
        plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[driv_index], label = drivers_dim[driv_index])

ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("QUALYFING TIME DIFERENCE") 
plt.xlabel("GP")
plt.ylabel("Distance to pole in ms")
plt.xticks(range(0, int(len(n_gps))), labels = n_gps, rotation=90)
plt.yticks(range(0, int(maxValuePlot(qualiDiferenceOrdered)) + 100, lenTicks(0, maxValuePlot(qualiDiferenceOrdered))))

#RED LINE LIMIT
x = [0, len(n_gps)]
y = [6000, 6000]
plt.plot(x, y, color = 'r',  label = 'OUT OF LIMITS')
yticks = plt.gca().get_yticks()
yticklabels = plt.gca().get_yticklabels()
#Label Modification
yticks[-1] = 6000
for i,num in enumerate(yticks):
    yticklabels[i] = str(num)
yticklabels[-1] = "OUT OF LIMIT"
plt.yticks(yticks, yticklabels)
###############

plt.show()

__Qualyfing Time Difference per Team__

In [ ]:
x_axes = [i for i in range(len(n_gps))]
for team_index, team in enumerate(qualiTeamDiference.T):
    plt.plot(x_axes, team, linewidth=2 , color = colorsTeams[team_index], label = teams[team_index]) 

ax = plt.gca()
ax.legend(loc=0, bbox_to_anchor=(1, 0.75))
ax.set_facecolor('#DFF0FF')
plt.title('QUALYFING TIME DIFFERENCE') 
plt.xlabel("GP")
plt.ylabel("Distance to pole in ms")
plt.xticks(range(0, int(len(n_gps))), labels = n_gps, rotation=90)
plt.yticks(range(0, int(maxValuePlot(qualiTeamDiference)) + 100, lenTicks(0, maxValuePlot(qualiTeamDiference))))

#RED LINE LIMIT
x = [0, len(n_gps)]
y = [6000, 6000]
plt.plot(x, y, color = 'r',  label = 'OUT OF LIMITS')
yticks = plt.gca().get_yticks()
yticklabels = plt.gca().get_yticklabels()
#Label Modification
yticks[-1] = 6000
for i,num in enumerate(yticks):
    yticklabels[i] = str(num)
yticklabels[-1] = "OUT OF LIMIT"
plt.yticks(yticks, yticklabels)
###############

plt.show()

__Qualyfing Time Difference per Team in %__

In [ ]:
x_axes = [i for i in range(len(n_gps))]
for team_index, team in enumerate(qualiTeamDiferencePerc.T):
    plt.plot(x_axes, team, linewidth=2 , color = colorsTeams[team_index], label = teams[team_index]) 

ax = plt.gca()
ax.legend(loc=0, bbox_to_anchor=(1, 0.75))
plt.title('QUALYFING TIME DIFFERENCE in %') 
plt.xlabel("GP")
plt.ylabel("Distance to pole in %")
plt.xticks(range(0, int(len(n_gps))), labels = n_gps, rotation=90)
plt.yticks(range(100, 108, 1))

#RED LINE LIMIT
x = [0, len(n_gps)]
y = [107, 107]
plt.plot(x, y, color = 'r',  label = 'OUT OF LIMITS')
yticks = plt.gca().get_yticks()
yticklabels = plt.gca().get_yticklabels()
#Label Modification
yticks[-1] = 107
for i,num in enumerate(yticks):
    yticklabels[i] = str(num)
yticklabels[-1] = "OUT OF LIMIT"
plt.yticks(yticks, yticklabels)
###############
plt.show()

Data Preprocessing:

In [ ]:
yearsProg = list(df["Year"].unique()) #Years Available
driversProg = list(df["Driver"].unique()) #Drivers Available
teamsProg = list(df["Team"].unique()) #Teams Available
totSumPoints = np.zeros((len(ses_points),len(yearsProg))) #Total Points in a Year


#Final Lists to Plot
finalReturn = np.zeros((5,len(driversProg),len(yearsProg))) #5 Sesions, X Drivers and Y total Years up to last 10?
finalReturnTeam = np.zeros((5,len(teamsProg),len(yearsProg))) #5 Sesions, X Teams and Y total Years up to last 10?
colorsAll = [0 for _ in range(len(driversProg))]
colorsTeamAll = [0 for _ in range(len(teamsProg))]

def updateColors(drivers:list, driversProg:list, colorsAll:list):
    colors = multidata.f1_teams(drivers, year, 'Default', 'Default').color  #Colors of each team deppending 
    for dIndex,driv in enumerate(driversProg):
        if driv in drivers:
            indexColor = drivers.index(driv)
            colorsAll[dIndex] = colors[indexColor]
    return colorsAll

def updateColorsTeam(teams: list, colors:list, teamsProg: list):
    colorsTeamAll = []
    alreadySelected = []
    for tIndex,team in enumerate(teamsProg):
        if team not in alreadySelected:
            alreadySelected.append(team)
            colIndex = teams.index(team)
            colorsTeamAll.append(colors[colIndex])
    
    return colorsTeamAll

def updateDataYear(yIndex: int,totalPoints: np.ndarray, drivers: list, finalReturn: np.ndarray, driversProg: list):
    sesions =  totalPoints.shape[0]
    for s in range(sesions):
        for dIndex, driver in enumerate(drivers):
            indexFinal = driversProg.index(driver)
            finalReturn[s,indexFinal,yIndex] = totalPoints[s,dIndex]

    return finalReturn

def updateDataTeam(yIndex: int,totalPoints: np.ndarray, teams: list, finalReturnTeam: np.ndarray, teamsProg: list):
    sesions =  totalPoints.shape[0]
    teams = multidata.lastTeamName(teams).correct
    for s in range(sesions):
        alreadySelected = []
        for tIndex,team in enumerate(teamsProg):
            alreadySelected.append(team)
            indexFinal = [t for t,tm in enumerate(teams) if tm == team]
            totalTeamSes = sum([totalPoints[s,i] for i in indexFinal])
            finalReturnTeam[s,tIndex,yIndex] = totalTeamSes

    return finalReturnTeam


for yIndex,year in enumerate(yearsProg):
    df_Year = df[ df['Year'] == year]
    drivers = list(df_Year["Driver"].unique()) #Drivers participating in the corresponding time period
    n_gps = list(df_Year["GP"].unique()) #GPs during the corresponding time period

    colorsAll = updateColors(drivers, driversProg, colorsAll)
    
    #Times in each GP and each driver and the corresponding points obtained during that period
    fp1_times = np.zeros((len(n_gps),len(drivers)))
    fp1_points = np.zeros((len(n_gps),len(drivers)))

    fp2_times = np.zeros((len(n_gps),len(drivers)))
    fp2_points = np.zeros((len(n_gps),len(drivers)))

    fp3_times = np.zeros((len(n_gps),len(drivers)))
    fp3_points = np.zeros((len(n_gps),len(drivers)))

    quali_times = np.zeros((len(n_gps),len(drivers)))
    quali_points = np.zeros((len(n_gps),len(drivers)))

    race_pos = np.zeros((len(n_gps),len(drivers)))
    race_points = np.zeros((len(n_gps),len(drivers)))

    yearData = df_Year.to_numpy()
    driver_name = colNames.index("Driver")
    gp_index = colNames.index("GP")

    fp1 = colNames.index("fp1_fastest_lap")
    fp2 = colNames.index("fp2_fastest_lap")
    fp3 = colNames.index("fp3_fastest_lap")
    quali = colNames.index("quali_fastest_lap")
    race = colNames.index("race_Result")


    for row in yearData:
        gpName = row[gp_index] #GP on this row
        driv = row[driver_name] #Driver on this row

        i = n_gps.index(gpName)
        j = drivers.index(driv)

        fp1_times[i,j] = row[fp1]
        fp2_times[i,j] = row[fp2]
        fp3_times[i,j] = row[fp3]
        quali_times[i,j] = row[quali]
        if row[race] == 'DNF': race_pos[i,j] = 0
        else:  race_pos[i,j] = row[race]

    points = [25,18,15,12,10,8,6,4,2,1,0,0,0,0,0,0,0,0,0,0]

    sesion = [fp1_times.copy(),fp2_times.copy(),fp3_times.copy(),quali_times.copy(), race_pos.copy()] #Include every Sesion time (4, len(gPs), len(drivers))
    ses_points = [fp1_points.copy(),fp2_points.copy(),fp3_points.copy(),quali_points.copy(), race_points.copy()] #Include every Sesion points (4, len(gPs), len(drivers))

    for ses_index, s in enumerate(sesion):
        for circ_index, circuit in enumerate(s):
            p = -1
            if sum(circuit) > 10:
                while circuit[np.argmin(circuit)] < sys.maxsize:
                    minim = np.argmin(circuit)
                    if circuit[minim] == 0:
                        circuit[minim] = sys.maxsize
                    else:
                        p += 1
                        minim =np.argmin(circuit)
                        circuit[minim] = sys.maxsize
                        ses_points[ses_index][circ_index,minim] = points[p]

    totalPoints = np.zeros((len(ses_points),len(drivers))) #Reduce the total to (4, len(driveres)) matrix which incluedes the total amount of points scored by each driver in each Sesion

    for ses_index, s in enumerate(ses_points):
        for driv_index, driver in enumerate(drivers):
            totalPoints[ses_index,driv_index] = sum(s[circ_index,driv_index] for circ_index, circuit in enumerate(s))
    
    for ses_index, s in enumerate(ses_points):
        totSumPoints[ses_index,yIndex] = np.sum(totalPoints[ses_index])

    finalReturn = updateDataYear(yIndex, totalPoints, drivers, finalReturn, driversProg)

    teams =  multidata.f1_teams(drivers, year, 'Default', 'Default').equipos
    finalReturnTeam = updateDataTeam(yIndex, totalPoints, teams, finalReturnTeam, teamsProg)
    if year == yearsProg[-1]:
        colors =  multidata.f1_teams(drivers, year, 'Default', 'Default').color
        colorsTeamAll = updateColorsTeam(teams,colors,teamsProg)

finalReturnAbsolute = np.zeros_like(finalReturn)
for s in range(finalReturn.shape[0]): #Session
    for d in range(finalReturn.shape[1]): #Driver
        finalReturnAbsolute[s,d] = np.cumsum(finalReturn[s,d])

finalReturnTeamAbsolute = np.zeros_like(finalReturnTeam)
for s in range(finalReturnTeam.shape[0]): #Session
    for t in range(finalReturnTeam.shape[1]): #Driver
        finalReturnTeamAbsolute[s,t] = np.cumsum(finalReturnTeam[s,t])

finalReturnPerc = np.zeros_like(finalReturn)
for s in range(finalReturn.shape[0]): #Session
    for y in range(finalReturn.shape[2]): #Year
        for d in range(finalReturn.shape[1]): #Driver
            finalReturnPerc[s,d,y] = finalReturn[s,d,y]*100/totSumPoints[s,y]

finalReturnTeamPerc = np.zeros_like(finalReturnTeam)
for s in range(finalReturnTeam.shape[0]): #Session
    for y in range(finalReturnTeam.shape[2]): #Year
        for d in range(finalReturnTeam.shape[1]): #Driver
            finalReturnTeamPerc[s,d,y] = finalReturnTeam[s,d,y]*100/totSumPoints[s,y]

__GENERAL PLOT SETTINGS__

In [ ]:
#### PLOT SETTINGS BY YEAR ####
import matplotlib.pyplot as plt
# backscreen, legend = multidata.plotColors(yearsProg[-1]).color
%matplotlib inline 
plt.rcParams["figure.figsize"] = (10.4, 8.8)
plt.rcParams["axes.facecolor"] =  '#DFF0FF'
plt.rcParams["legend.facecolor"] = '#DFF0FF'
plt.rcParams["legend.edgecolor"] = '#000000'
################################

__Drivers Absolute Progression by Year Plot__

In [ ]:
def orderFRA(finalReturnAbsolute: np.ndarray, driversProg: list, colorsAll:list,maxNumItems = 20):
    orderedArray = np.zeros((finalReturnAbsolute.shape[0],maxNumItems,finalReturnAbsolute.shape[2]))
    driversPlot = []
    colorsPlot = []
    for s in range(finalReturnAbsolute.shape[0]): #Session
        maxSesionTotal = np.zeros(finalReturnAbsolute.shape[1])
        for d in range(finalReturnAbsolute.shape[1]): #Driver
            maxSesionTotal[d] = finalReturnAbsolute[s,d,-1] #Total Last Year
        driversSes = []
        colorsSes = []
        for i in range(maxNumItems): #Best 25 Drivers
            max_value = np.argmax(maxSesionTotal)
            orderedArray[s,i] += finalReturnAbsolute[s,max_value]
            driversSes.append(driversProg[max_value])
            colorsSes.append(colorsAll[max_value])

            maxSesionTotal[max_value] = -1 #To not re-elect same driver
        
        driversPlot.append(driversSes)
        colorsPlot.append(colorsSes)
    
    return orderedArray, driversPlot, colorsPlot

finalReturnAbsoluteOrd, driversPlot, colorsPlot = orderFRA(finalReturnAbsolute, driversProg, colorsAll)


x_axes = [i for i in range(len(yearsProg))]
titles = ['FP1 YEAR PROGRESSION', 'FP2 YEAR PROGRESSION','FP3 YEAR PROGRESSION', 'QUALIFYING YEAR PROGRESSION', 'RACE YEAR PROGRESSION']
for s_index,sesion in enumerate(finalReturnAbsoluteOrd):
    # teams_av = []
    # teams_driv = multidata.f1_teams(driversPlot[s_index], year, 'Default', 'Default').equipos
    drivers_dim = driv_min(driversPlot[s_index].copy())
    for driv_index, driver in enumerate(sesion):
        # if teams_driv[driv_index] not in teams_av:
        #     teams_av.append(teams_driv[driv_index])
        plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[s_index][driv_index], 
        label = drivers_dim[driv_index] +' ('+str(finalReturnAbsoluteOrd[s_index][driv_index][-1]) +')') 
        # else:
        #     plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[s_index][driv_index], label =drivers_dim[driv_index])


    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(titles[s_index]) 
    plt.xlabel("Year")
    plt.ylabel("Total Points")
    plt.xticks(range(0, int(len(yearsProg))), labels = yearsProg, rotation=90)
    plt.yticks(range(int(minValuePlot(sesion) - 25), int(maxValuePlot(sesion) + 25), lenTicks(minValuePlot(sesion) - 25, maxValuePlot(sesion) + 25)))
    plt.show()

__Teams Absolute Progression by Year Plot__

In [ ]:
finalReturnTeamsAbsoluteOrd, teamsPlot, colorsPlot = orderFRA(finalReturnTeamAbsolute, teamsProg, colorsTeamAll, maxNumItems = len(teamsProg))

x_axes = [i for i in range(len(yearsProg))]
titles = ['FP1 YEAR PROGRESSION', 'FP2 YEAR PROGRESSION','FP3 YEAR PROGRESSION', 'QUALIFYING YEAR PROGRESSION', 'RACE YEAR PROGRESSION']
for s_index,sesion in enumerate(finalReturnTeamsAbsoluteOrd):
    for tm_index, team in enumerate(sesion):
        plt.plot(x_axes, team, linewidth=2, color = colorsPlot[s_index][tm_index], 
        label = teamsPlot[s_index][tm_index] +' ('+str(finalReturnTeamsAbsoluteOrd[s_index][tm_index][-1]) +')') 


    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(titles[s_index]) 
    plt.xlabel("Year")
    plt.ylabel("Total Points")
    plt.xticks(range(0, int(len(yearsProg))), labels = yearsProg, rotation=90)
    plt.yticks(range(int(minValuePlot(sesion) - 25), int(maxValuePlot(sesion) + 25), lenTicks(minValuePlot(sesion) - 25, maxValuePlot(sesion) + 25)))
    plt.show()

__Drivers Progression Plot by Year in %__

In [ ]:
finalReturnAbsoluteOrd, driversPlot, colorsPlot = orderFRA(finalReturnPerc, driversProg, colorsAll)


x_axes = [i for i in range(len(yearsProg))]
titles = ['FP1 YEAR PROGRESSION', 'FP2 YEAR PROGRESSION','FP3 YEAR PROGRESSION', 'QUALIFYING YEAR PROGRESSION', 'RACE YEAR PROGRESSION']
for s_index,sesion in enumerate(finalReturnAbsoluteOrd):
    teams_av = []
    teams_driv = multidata.f1_teams(driversPlot[s_index], year, 'Default', 'Default').equipos
    drivers_dim = driv_min(driversPlot[s_index].copy())
    for driv_index, driver in enumerate(sesion):
        if teams_driv[driv_index] not in teams_av:
            teams_av.append(teams_driv[driv_index])
            plt.plot(x_axes, driver, linewidth=2, color = colorsPlot[s_index][driv_index], 
            label = drivers_dim[driv_index] +' ('+str(round(finalReturnAbsoluteOrd[s_index][driv_index][-1],2)) +' %)') 
        else:
            plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[s_index][driv_index], 
            label =drivers_dim[driv_index] +' ('+str(round(finalReturnAbsoluteOrd[s_index][driv_index][-1],2)) +' %)')

    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(titles[s_index]) 
    plt.xlabel("Year")
    plt.ylabel("Total Percentage")
    plt.xticks(range(0, int(len(yearsProg))), labels = yearsProg, rotation=90)
    plt.yticks(range(0, int(maxValuePlot(sesion) + 3), lenTicks(0, maxValuePlot(sesion) + 3)))
    plt.show()

__Teams Progression Plot by Year in %__

In [ ]:
finalReturnTeamsAbsoluteOrd, teamsPlot, colorsPlot = orderFRA(finalReturnTeamPerc, teamsProg, colorsTeamAll, maxNumItems = len(teamsProg))

x_axes = [i for i in range(len(yearsProg))]
titles = ['FP1 YEAR PROGRESSION', 'FP2 YEAR PROGRESSION','FP3 YEAR PROGRESSION', 'QUALIFYING YEAR PROGRESSION', 'RACE YEAR PROGRESSION']
for s_index,sesion in enumerate(finalReturnTeamsAbsoluteOrd):
    for tm_index, team in enumerate(sesion):
        plt.plot(x_axes, team, linewidth=2, color = colorsPlot[s_index][tm_index], 
        label = teamsPlot[s_index][tm_index] +' ('+str(round(finalReturnTeamsAbsoluteOrd[s_index][tm_index][-1],2)) +')') 


    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(titles[s_index]) 
    plt.xlabel("Year")
    plt.ylabel("Total Percentage")
    plt.xticks(range(0, int(len(yearsProg))), labels = yearsProg, rotation=90)
    plt.yticks(range(0, int(maxValuePlot(sesion) + 5), lenTicks(0, maxValuePlot(sesion) + 5)))
    plt.show()

__Car Difference between 2 different years (Only Qualyfing)__

In [ ]:
import re

y1 = 2021
y2 = 2022

#Obtain Qualy Times
df_y1 = df.loc[df['Year'] == y1]
df_y2 = df.loc[df['Year'] == y2]

drivers_y1 = list(df_y1["Driver"].unique()) #Drivers participating in the corresponding time period
teams_y1 = multidata.f1_teams(drivers_y1, y1, 'Default', 'Default').equipos  #F1 Teams participating in the corresponding time period
teams_y1 = list(set(teams_y1))
n_gps_y1 = list(df_y1["GP"].unique()) #GPs during the corresponding time period

# Define regular expression pattern to match digits
pattern = '\d+'

# Create a new list without numbers
n_gps_y1 = [re.sub(pattern, '', s) for s in n_gps_y1]

drivers_y2 = list(df_y2["Driver"].unique()) #Drivers participating in the corresponding time period
teams_y2 = multidata.f1_teams(drivers_y2, y2, 'Default', 'Default').equipos #F1 Teams participating in the corresponding time period
teams_y2 = list(set(teams_y2))
n_gps_y2 = list(df_y2["GP"].unique()) #GPs during the corresponding time period

bothGP = []
for elem in  n_gps_y2:
    if elem in n_gps_y1 and elem not in bothGP:
        bothGP.append(elem) 
        
times_y1 = np.zeros((len(bothGP),len(drivers_y1)))
times_y2 = np.zeros((len(bothGP),len(drivers_y2)))

pd.options.mode.chained_assignment = None  # !AVOID WARNING
df_y1['GP'] = df_y1['GP'].apply(lambda x: re.sub('1', '', x))
df_y2['GP'] = df_y2['GP'].apply(lambda x: re.sub('1', '', x))

all_y1 = df_y1.to_numpy()

driver_name = colNames.index("Driver")
gp_index = colNames.index("GP")
quali = colNames.index("quali_fastest_lap")

for row in all_y1:
    gpName = row[gp_index] #GP on this row
    driv = row[driver_name] #Driver on this row

    if gpName in bothGP:
        i = bothGP.index(gpName)
        j = drivers_y1.index(driv)
        times_y1[i,j] = row[quali]    

all_y2 = df_y2.to_numpy()

driver_name = colNames.index("Driver")
gp_index = colNames.index("GP")
quali = colNames.index("quali_fastest_lap")

for row in all_y2:
    gpName = row[gp_index] #GP on this row
    driv = row[driver_name] #Driver on this row
    
    if gpName in bothGP:
        i = bothGP.index(gpName)
        j = drivers_y2.index(driv)
        times_y2[i,j] = row[quali]    

In [ ]:
quali_Team_y1 = np.zeros((len(bothGP), len(teams_y1)))
quali_Team_y2 = np.zeros((len(bothGP), len(teams_y2)))

#Transform Teams Name to Equalize
cor_t1 = multidata.lastTeamName(teams_y1.copy()).correct
cor_t2 = multidata.lastTeamName(teams_y2.copy()).correct

bothTeam = []
for elem in  cor_t1:
    if elem in cor_t2 and elem not in bothTeam:
        bothTeam.append(elem) 

finaltoPlot = np.zeros((len(bothGP), len(bothTeam)))
        
for gp_index,gp in enumerate(bothGP):
    # Obtain array of square of each element in x
    #Drivers
    negArray_y1 = my_func(times_y1[gp_index])
    negArray_y2 = my_func(times_y2[gp_index])
    

    #Teams
    quali_times_y1, teams_y1 = qualiTeamsFun(negArray_y1, drivers_y1, teams_y1, y1)
    quali_times_y2, teams_y2 = qualiTeamsFun(negArray_y2, drivers_y2, teams_y2, y2)

    for tIndex, team in enumerate(bothTeam):
        finaltoPlot[gp_index,tIndex] =  (quali_times_y2[cor_t2.index(team)]/quali_times_y1[cor_t1.index(team)])*100


In [ ]:
def removeOutliers2Side(qualiDiference: np.ndarray, drivers: list, uplimit: int, lowlimit: int):
    newDrivers = []
    removedIndeces = []
    qualiDiference[(qualiDiference >= uplimit) & (qualiDiference != np.inf)] = uplimit #Over limit is understood as an outlier and therefore it does not count.
    qualiDiference[qualiDiference <= lowlimit] = lowlimit #Under limit is understood as an outlier and therefore it does not count.
    for dIndex,driv in enumerate(qualiDiference.T):
        driv = driv.copy()
        driv[driv != np.inf] = False
        driv[driv == np.inf] = True
        if sum(driv) < len(driv):
            newDrivers.append(drivers[dIndex])
        else:
            removedIndeces.append(dIndex)

    qualiDiference = np.delete(qualiDiference, removedIndeces, axis=1)

    return qualiDiference, newDrivers


# *DIFICULT TO ADJUST, DO IT BY OUR SELVES
uplimit = 104
lowlimit = 97

finaltoPlot, teamsNotInf = removeOutliers2Side(finaltoPlot, bothTeam, uplimit, lowlimit)
colorsTeams = multidata.lastTeamName(teamsNotInf.copy()).color

x_axes = [i for i in range(len(bothGP))]
for team_index, team in enumerate(finaltoPlot.T):
    plt.plot(x_axes, team, linewidth=2 , color = colorsTeams[team_index], label = teamsNotInf[team_index]) 

ax = plt.gca()
ax.legend(loc=0, bbox_to_anchor=(1, 0.75))
ax.set_facecolor('#DFF0FF')
plt.title(f'DIFFERENCE IN QUALY TIME FROM {y1} TO {y2}') 
plt.xlabel("GP")
plt.ylabel("Percentage Difference")
plt.xticks(range(0, int(len(bothGP))), labels = bothGP, rotation=90)
plt.yticks(range(lowlimit, uplimit + 1, 1))

#RED LINE LIMIT
x = [0, len(bothGP)]
y = [lowlimit, lowlimit]
plt.plot(x, y, color = 'r',  label = 'OUT OF LIMITS')
y = [uplimit, uplimit]
plt.plot(x, y, color = 'r',  label = 'OUT OF LIMITS')
yticks = plt.gca().get_yticks()
yticklabels = plt.gca().get_yticklabels()
#Label Modification
yticks[-1] = uplimit
for i,num in enumerate(yticks):
    yticklabels[i] = str(num)
yticklabels[0] = "OUT OF LIMIT"
yticklabels[-1] = "OUT OF LIMIT"
plt.yticks(yticks, yticklabels)
###############
plt.show()